In [26]:
import pandas as pd
import numpy as np
import pickle
import time 
import os
from datetime import datetime
from operator import itemgetter

data_folder_path = '../'
raw_data_path = "../raw_data"
storage_path = "../database_storage"
daily_path = "../daily"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
DCBS_path = os.path.join(storage_path, "DCBS.pkl")

start_time = time.time()

with open('database_ByZCG\\DTBS_DataFrame.pkl', 'rb') as f:  
    DTBS_DataFrame = pickle.load(f)
    
# 只提取A区中存活的转债的时间序列信息
A_DF = DTBS_DataFrame['A']
A_DF = A_DF[A_DF['ia']==1].copy()
B_DF = DTBS_DataFrame['B'].copy()
D_DF = DTBS_DataFrame['D'].copy()

In [17]:
# 设置周一（或其他时间）换仓
start_day = '2023-01-01'
end_day = '2023-07-31'

set_which_day = 'thu'
name = '热门可转债+低价程度（全量数据DTBS，用于回测）.xlsx'

"""
按需选择需要分析的策略：
('低价小市值5（全量数据DTBS，用于回测）.xlsx', 'mon')
('高ytm（全量数据DTBS，用于回测）.xlsx', 'wed')
('区间双低3（全量数据DTBS，用于回测）.xlsx', 'thu')
('双低5（全量数据DTBS，用于回测）.xlsx', 'thu')
('小市值3双低5（全量数据DTBS，用于回测）.xlsx', 'wed')
('热门可转债+双低（全量数据DTBS，用于回测）.xlsx', 'mon')
('低价5（全量数据DTBS，用于回测）.xlsx', 'mon')
"""


df = pd.read_excel('pre_data_for_backtest\\'+ name)
print(df)
trade_date_list = sorted(list(D_DF.loc[(D_DF['date'] >= start_day)&(D_DF['date'] <= end_day)&(D_DF['which_day'] == set_which_day),'date']))
print(trade_date_list)

output_df = df[df['日期'].isin(trade_date_list[0:-1])][['可转债代码', '日期', '转股溢价率', '债券余额（亿）', '换手率', '到期收益率', '剩余期限（年）', '双低', 'qs' ,'qs30']].copy()
output_df.rename(columns={'日期' : '买入日期'}, inplace=True)
output_df['卖出日期'] = output_df['买入日期'].apply(lambda x:trade_date_list[trade_date_list.index(x)+1])
output_df['持仓周期'] = output_df['买入日期'] + '至' + output_df['卖出日期']
temp = pd.merge(output_df,B_DF,left_on = '可转债代码',right_on='bond_code',how='left')
output_df['可转债名称'] = list(temp['cn'])
output_df['正股代码'] = list(temp['sc'])
output_df['正股名称'] = list(temp['sn'])
output_df['买入价'] = list(pd.merge(output_df,A_DF[['bond_code', 'date', 'dp']],left_on = ['可转债代码', '买入日期'],right_on=['bond_code', 'date'],how='left')['dp'])
output_df['卖出价'] = list(pd.merge(output_df,A_DF[['bond_code', 'date', 'dp']],left_on = ['可转债代码', '卖出日期'],right_on=['bond_code', 'date'],how='left')['dp'])
output_df['收益率'] = output_df['卖出价'] / output_df['买入价'] - 1
output_df['买卖价差'] = output_df['卖出价'] - output_df['买入价']
# 使用transform方法计算每个分组内的总和
grouped_sum = output_df.groupby('买入日期')['买卖价差'].transform('sum')

# 计算每个分组的百分比
output_df['对于整体收益率的贡献'] = output_df['买卖价差'] / grouped_sum

output_df = output_df[['持仓周期', '买入日期', '卖出日期', '可转债代码', '可转债名称', '正股代码', '正股名称', '买入价', '卖出价', '收益率', 
                       '对于整体收益率的贡献', '转股溢价率', '债券余额（亿）', '换手率', '到期收益率', '剩余期限（年）', '双低', 'qs' ,'qs30']].copy()
print(output_df)

strategy_name = str(name[ : name.index('（')])
mapping = {'mon' : '周一', 
           'tue' : '周二', 
           'wed' : '周三', 
           'thu' : '周四', 
           'fri' : '周五'}
output_excel_name = strategy_name + '（换仓日：' + mapping[set_which_day] + '，用于分析具体转债对整体收益率的影响）.xlsx'
output_df.to_excel('data_for_analyze\\' + output_excel_name, index=False)

end_time = time.time()
run_time = end_time - start_time
# 输出运行时间
print(f"程序运行时间：{run_time:.2f}秒")

              日期      可转债代码  价格（收盘价全价）  正股名称       涨跌幅        双低    转股溢价率  \
0     2020-01-17  113537.SH    123.070  文灿股份 -0.009417  127.6664   4.5964   
1     2020-01-17  123033.SZ    113.000  金力永磁 -0.002648  132.4050  19.4050   
2     2020-01-17  127013.SZ    126.113  创维数字 -0.008694  127.0210   0.9080   
3     2020-01-17  128025.SZ    115.853  特一药业 -0.002986  117.6692   1.8162   
4     2020-01-20  123012.SZ    120.875  万顺新材 -0.000041  127.9643   7.0893   
...          ...        ...        ...   ...       ...       ...      ...   
3609  2023-07-27  123035.SZ    127.709   利亚德 -0.006388  165.6590  37.9500   
3610  2023-07-27  123131.SZ    128.101  奥飞数据 -0.008859  167.5310  39.4300   
3611  2023-07-27  123163.SZ    123.448  金沃股份 -0.000421  153.9280  30.4800   
3612  2023-07-27  123175.SZ    123.145  百川畅银 -0.001443  161.7150  38.5700   
3613  2023-07-27  128033.SZ    126.080   雪迪龙 -0.004736  146.3500  20.2700   

       债券余额（亿）   剩余期限（年）       换手率    到期收益率  qs  qs30  
0     5.691400  5.3

In [29]:
# 查看所有换仓日的情况
start_day = '2023-05-01'
end_day = '2023-08-07'

set_which_days = ['mon', 'tue', 'wed', 'thu', 'fri']

name = '热门可转债+低价程度（全量数据DTBS，用于回测）.xlsx'

"""
按需选择需要分析的策略：
('低价小市值5（全量数据DTBS，用于回测）.xlsx', 'mon')
('高ytm（全量数据DTBS，用于回测）.xlsx', 'wed')
('区间双低3（全量数据DTBS，用于回测）.xlsx', 'thu')
('双低5（全量数据DTBS，用于回测）.xlsx', 'thu')
('小市值3双低5（全量数据DTBS，用于回测）.xlsx', 'wed')
('热门可转债+双低（全量数据DTBS，用于回测）.xlsx', 'mon')
('低价5（全量数据DTBS，用于回测）.xlsx', 'mon')
"""
mapping = {'mon' : '周一', 
           'tue' : '周二', 
           'wed' : '周三', 
           'thu' : '周四', 
           'fri' : '周五'}

df = pd.read_excel('pre_data_for_backtest\\'+ name)
print(df)

all_output_df = []

for set_which_day in set_which_days:
    
    trade_date_list = sorted(list(D_DF.loc[(D_DF['date'] >= start_day)&(D_DF['date'] <= end_day)&(D_DF['which_day'] == set_which_day),'date']))
    print(trade_date_list)

    output_df = df[df['日期'].isin(trade_date_list[0:-1])][['可转债代码', '日期', '转股溢价率', '债券余额（亿）', '换手率', '到期收益率', '剩余期限（年）', '双低', 'qs' ,'qs30']].copy()
    output_df.rename(columns={'日期' : '买入日期'}, inplace=True)
    output_df['卖出日期'] = output_df['买入日期'].apply(lambda x:trade_date_list[trade_date_list.index(x)+1])
    output_df['持仓周期'] = output_df['买入日期'] + '至' + output_df['卖出日期']
    temp = pd.merge(output_df,B_DF,left_on = '可转债代码',right_on='bond_code',how='left')
    output_df['可转债名称'] = list(temp['cn'])
    output_df['正股代码'] = list(temp['sc'])
    output_df['正股名称'] = list(temp['sn'])
    output_df['买入价'] = list(pd.merge(output_df,A_DF[['bond_code', 'date', 'dp']],left_on = ['可转债代码', '买入日期'],right_on=['bond_code', 'date'],how='left')['dp'])
    output_df['卖出价'] = list(pd.merge(output_df,A_DF[['bond_code', 'date', 'dp']],left_on = ['可转债代码', '卖出日期'],right_on=['bond_code', 'date'],how='left')['dp'])
    output_df['收益率'] = output_df['卖出价'] / output_df['买入价'] - 1
    output_df['买卖价差'] = output_df['卖出价'] - output_df['买入价']
    # 使用transform方法计算每个分组内的总和
    grouped_sum = output_df.groupby('买入日期')['买卖价差'].transform('sum')

    # 计算每个分组的百分比
    output_df['对于整体收益率的贡献'] = output_df['买卖价差'] / grouped_sum
    
    output_df['换仓日'] = mapping[set_which_day]
    
    output_df = output_df[['换仓日', '持仓周期', '买入日期', '卖出日期', '可转债代码', '可转债名称', '正股代码', '正股名称', '买入价', '卖出价', '收益率', 
                           '对于整体收益率的贡献', '转股溢价率', '债券余额（亿）', '换手率', '到期收益率', '剩余期限（年）', '双低', 'qs' ,'qs30']].copy()
    
    
    
    ouput_last_df = df[df['日期']==trade_date_list[-1]][['可转债代码', '日期', '转股溢价率', '债券余额（亿）', '换手率', '到期收益率', '剩余期限（年）', '双低', 'qs' ,'qs30']].copy()
    ouput_last_df.rename(columns={'日期' : '买入日期'}, inplace=True)
    temp = pd.merge(ouput_last_df,B_DF,left_on = '可转债代码',right_on='bond_code',how='left')
    ouput_last_df['可转债名称'] = list(temp['cn'])
    ouput_last_df['正股代码'] = list(temp['sc'])
    ouput_last_df['正股名称'] = list(temp['sn'])
    ouput_last_df['买入价'] = list(pd.merge(ouput_last_df,A_DF[['bond_code', 'date', 'dp']],left_on = ['可转债代码', '买入日期'],right_on=['bond_code', 'date'],how='left')['dp'])
    ouput_last_df['换仓日'] = mapping[set_which_day]
    
    output_df = pd.concat([output_df, ouput_last_df], ignore_index=True)
    
    all_output_df.append(output_df)
    print(output_df)

output_df = pd.concat(all_output_df, ignore_index=True)



output_df.sort_values(by=['买入日期','可转债代码'],ascending=[True,True] ,inplace=True)

strategy_name = str(name[ : name.index('（')])

output_excel_name = strategy_name + '（换仓日：周一至周五，用于分析具体转债对整体收益率的影响）.xlsx'
output_df.to_excel('data_for_analyze\\' + output_excel_name, index=False)

end_time = time.time()
run_time = end_time - start_time
# 输出运行时间
print(f"程序运行时间：{run_time:.2f}秒")

              日期      可转债代码  价格（收盘价全价）  正股名称       涨跌幅        双低    转股溢价率  \
0     2020-01-17  113537.SH    123.070  文灿股份 -0.009417  127.6664   4.5964   
1     2020-01-17  123033.SZ    113.000  金力永磁 -0.002648  132.4050  19.4050   
2     2020-01-17  127013.SZ    126.113  创维数字 -0.008694  127.0210   0.9080   
3     2020-01-17  128025.SZ    115.853  特一药业 -0.002986  117.6692   1.8162   
4     2020-01-20  123012.SZ    120.875  万顺新材 -0.000041  127.9643   7.0893   
...          ...        ...        ...   ...       ...       ...      ...   
3638  2023-08-07  123004.SZ    113.230  节能铁汉 -0.016887  120.7300   7.5000   
3639  2023-08-07  123198.SZ    123.998  金埔园林 -0.023630  149.5380  25.5400   
3640  2023-08-07  128017.SZ    122.834  金禾实业 -0.036860  131.9640   9.1300   
3641  2023-08-07  128023.SZ    118.850  亚太股份 -0.007623  138.4000  19.5500   
3642  2023-08-07  128123.SZ    122.880  国光股份 -0.005825  161.5700  38.6900   

       债券余额（亿）   剩余期限（年）        换手率    到期收益率  qs  qs30  
0     5.691400  5.

   换仓日                   持仓周期        买入日期        卖出日期      可转债代码 可转债名称  \
0   周三  2023-05-10至2023-05-17  2023-05-10  2023-05-17  110057.SH  现代转债   
1   周三  2023-05-10至2023-05-17  2023-05-10  2023-05-17  111002.SH  特纸转债   
2   周三  2023-05-10至2023-05-17  2023-05-10  2023-05-17  111012.SH  福新转债   
3   周三  2023-05-10至2023-05-17  2023-05-10  2023-05-17  113588.SH  润达转债   
4   周三  2023-05-10至2023-05-17  2023-05-10  2023-05-17  113665.SH  汇通转债   
..  ..                    ...         ...         ...        ...   ...   
58  周三  2023-07-26至2023-08-02  2023-07-26  2023-08-02  123035.SZ  利德转债   
59  周三  2023-07-26至2023-08-02  2023-07-26  2023-08-02  123163.SZ  金沃转债   
60  周三                    NaN  2023-08-02         NaN  118034.SH  晶能转债   
61  周三                    NaN  2023-08-02         NaN  123107.SZ  温氏转债   
62  周三                    NaN  2023-08-02         NaN  128017.SZ  金禾转债   

         正股代码  正股名称      买入价      卖出价       收益率  对于整体收益率的贡献  转股溢价率  债券余额（亿）  \
0   600420.SH  国药现代  127.445  12